In [1]:
# Dependencies
import os 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress 

#Reading in Files
cdc_df = pd.read_csv("Resource/VSRR_Provisional_Drug_Overdose_Death_Counts_2015_2021.csv")
care_df = pd.read_csv("Resource/Medicare_Opioid_Prescribing_Data_2013_2019.csv", low_memory = False)
caid_df = pd.read_csv("Resource/Medicaid_Opioid_Prescribing_data_2013_2019.csv", low_memory = False)